In [55]:
import sys
parent_dir='C:\\Users\\garav\\AGOP\\DLR'
model_dir= 'C:\\Users\\garav\\AGOP\\DLR\\trained_models\\MNIST\\model4\\nn_models\\'
#parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [57]:
import torch
import torchvision
import torchvision.transforms as transforms
from utils import agop_conv as agc
from utils import trainer as tr
from torch.utils.data import Dataset
import random
import torch.backends.cudnn as cudnn
import rfm
import numpy as np
from trained_models.MNIST.model4 import trainer as t
import numpy as np
from sklearn.model_selection import train_test_split
from torch.linalg import norm
from torchvision import models
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from utils.groupy.gconv.pytorch_gconv.splitgconv2d import P4ConvZ2, P4ConvP4
from copy import deepcopy

In [59]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device='cpu'
print(f"Using device: {device}")


Using device: cuda:0


In [61]:
torch.cuda.empty_cache()

In [65]:
trainloader, valloader, testloader = t.get_loaders()
net= t.get_untrained_net()
init_net= deepcopy(net)
import os
if os.path.exists(model_dir+'mnist_gcnn_trained_nn.pth'):
    checkpoint = torch.load(model_dir+'mnist_gcnn_trained_nn.pth', map_location=torch.device(device))
    net.load_state_dict(checkpoint['state_dict'])  # Access the 'state_dict' within the loaded dictionary
    print("Model weights loaded successfully.")

#print("Train the network first")
#t.train_net()

Model weights loaded successfully.


In [16]:
tr.visualize_predictions(net, testloader, range(10), device, num_images=16)

C:\Users\garav\AGOP\DLR\trained_models\MNIST\model4\model4.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [73]:
''' This module does the following
1. Scan the network for conv layers
2. For each conv layer compute W^TW of eq 3
3. For each conv layer compute the AGOP(AJOP in case of multiple outputs)
4. For each conv layer print the pearson correlation between 2 and 3
'''

import torch
import torch.nn as nn
import random
import numpy as np
#from functorch import jacrev, vmap
from torch.func import jacrev
from torch.nn.functional import pad
#import dataset
from numpy.linalg import eig
from copy import deepcopy
from torch.linalg import norm, svd
from torchvision import models
import visdom
from torch.linalg import norm, eig


SEED = 2323

torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.cuda.manual_seed(SEED)

vis = visdom.Visdom('http://127.0.0.1', use_incoming_socket=False)
vis.close(env='main')

def patchify(x, patch_size, stride_size, padding=None, pad_type='zeros'):
    '''
        Given an input image (n,c,h,w) generate (n,w_out,h_out,c,q,s) respecting stride,padding, 
        w_out is number of pathces along the width for the given stride after padding
        h_out is number of pathces along the height for the given stride after padding
        (q,s) is the kernel dimensions 
    '''
    q1, q2 = patch_size
    s1, s2 = stride_size
    print("image shape",x)
    if padding is None:
        pad_1 = (q1-1)//2
        pad_2 = (q2-1)//2
    else:
        pad_1, pad_2 = padding

    pad_dims = (pad_2, pad_2, pad_1, pad_1)
    if pad_type == 'zeros':
        x = pad(x, pad_dims)
    elif pad_type == 'circular':
        x = pad(x, pad_dims, 'circular')
        
    patches = x.unfold(2, q1, s1).unfold(3, q2, s2) #(n, c, h_out, w_out, q, s)
    patches = patches.transpose(1, 3).transpose(1, 2) #(n,w_out,h_out,c,q,s) 
    return patches

class PatchConvLayer(nn.Module):
    def __init__(self, conv_layer):
        super().__init__()
        self.layer = conv_layer #(k,c,q,s)

    def forward(self, patches):
        out = torch.einsum('nwhcqr, kcqr -> nwhk', patches, self.layer.weight)
        n, w, h, k = out.shape
        out = out.transpose(1, 3).transpose(2, 3) #(n,k,w_out,h_out)
        return out

def get_jacobian(net, data, c_idx=0, chunk=100):
    with torch.no_grad():
        def single_net(x):
            # x is (w_out,h_out,c,q,s)
            return net(x.unsqueeze(0))[:,c_idx*chunk:(c_idx+1)*chunk].squeeze(0)
        # Parallelize across the images.
        return torch.vmap(jacrev(single_net))(data) #(n, chunk, w_out, h_out, c, q, s)

def egop(model, z):
    ajop = 0
    c = 10
    chunk_idxs = 1
    #Chunking is done to compute jacobian as chunks. This saves memory
    #TODO: chunk should be passed as argument
    chunk = c // chunk_idxs
    for i in range(chunk_idxs):
        J = get_jacobian(model, z, c_idx=i, chunk=chunk)
        n, c, w, h, _, _, _ = J.shape
        J = J.transpose(1, 3).transpose(1, 2) #(n, w_out, h_out, chunk, c, q, s)
        grads = J.reshape(n*w*h, c, -1) #(n*w_out*h_out, chunk, c*q*s)
        #Clarify: Where is mean taken
        ajop += torch.einsum('ncd, ncD -> dD', grads, grads) #(c*q*s,c*q*s)
    return ajop


def load_nn(net, init_net, layer_idx=0):
    '#TODO: Replace net with net.features'
    
    count = 0
    # Get the layer_idx+1 th conv layer
    for idx, m in enumerate(net.children()):
        if isinstance(m, P4ConvZ2):
            count += 1
        if count-1 == layer_idx:
            l_idx = idx
            break

    layers = list(net.children())
    layers_init= list(init_net.children())
    layer= layers[l_idx]
    layer_init = layers_init[l_idx]
    
    # Extract all the meta info of the current conv layer.
    (q, s) = layer.kernel_size
    (pad1, pad2) = layer.padding
    (s1, s2) = layer.stride
    
    # Extract W matrix
    M = layer.weight
    M0= layer_init.weight
    
    # Truncate all layers before l_idx and wrap the current conv layer as a PatchConvLayer class. 
    patchnet = deepcopy(net)
    patchnet = layers[l_idx:]
    layer = PatchConvLayer(layers[l_idx])
    patchnet[0] = layer
    

    k, ki, ip_stab, q,s= M.shape
            
    # Build W which is a (k, c*q*s) matrix. Check ip_stab usage
    M = M.reshape(-1, ki*ip_stab*q*s)
            
    # Compute WtW which is (c*q*s,c*q*s) matrix
    M = torch.einsum('nd, nD -> dD', M, M)
            
    # Build W0 from the untrained net which is a (k, c*q*s) matrix         
    M0 = M0.reshape(-1, ki*ip_stab*q*s)
    
    # Compute W0tW0 which is (c*q*s,c*q*s) matrix
    M0 = torch.einsum('nd, nD -> dD', M0, M0)

    return net, patchnet, M, M0, l_idx, [(q, s), (pad1,pad2), (s1,s2)]


def get_grads(net, patchnet, trainloader,
              kernel=(3,3), padding=(1,1),
              stride=(1,1), layer_idx=0):
    net.eval()
    net.cuda()
    patchnet.eval()
    patchnet.cuda()
    M = 0
    q, s = kernel
    pad1, pad2 = padding
    s1, s2 = stride

    # Num images for taking AGOP (Can be small for early layers)
    MAX_NUM_IMGS = 10

    for idx, batch in enumerate(trainloader):
        print("Computing GOP for sample " + str(idx) + \
              " out of " + str(MAX_NUM_IMGS))
        imgs, _ = batch
        with torch.no_grad():
            imgs = imgs.cuda()        
            # Run the first half of the network wrt to the current layer 
            imgs = net.features[:layer_idx](imgs).cpu() #(n,c,h,w)
        patches = patchify(imgs, (q, s), (s1,s2), padding=(pad1,pad2))#(n,w_out,h_out,c,q,s)
        patches = patches.cuda()
        #print(patches.shape)
        M += egop(patchnet, patches).cpu()
        del imgs, patches
        torch.cuda.empty_cache()
        if idx >= MAX_NUM_IMGS:
            break
    net.cpu()
    patchnet.cpu()
    return M


def min_max(M):
    return (M - M.min()) / (M.max() - M.min())


def correlation(M1, M2):
    M1 -= M1.mean()
    M2 -= M2.mean()

    norm1 = norm(M1.flatten())
    norm2 = norm(M2.flatten())

    return torch.sum(M1.cuda() * M2.cuda()) / (norm1 * norm2)


def verify_NFA(net, init_net, trainloader, layer_idx=0):


    net, patchnet, M, M0, l_idx, conv_vals = load_nn(net,
                                                     init_net,
                                                     layer_idx=layer_idx)
    (q, s), (pad1, pad2), (s1, s2) = conv_vals

    i_val = correlation(M0, M)
    print("Correlation between Initial and Trained CNFM: ", i_val)

    G = get_grads(net, patchnet, trainloader,
                  kernel=(q, s),
                  padding=(pad1, pad2),
                  stride=(s1, s2),
                  layer_idx=l_idx)
    print("Shpae after gradients: ", G.shape)
    G = sqrt(G)
    Gop = G.clone()
    r_val = correlation(M, G)
    print("Correlation between Trained CNFM and AGOP: ", r_val)
    print("Final: ", i_val, r_val)
    return Gop 
    #return i_val.data.numpy(), r_val.data.numpy()



def sqrt(G):
    U, s, Vt = svd(G)
    s = torch.pow(s, 1./2)
    G = U @ torch.diag(s) @ Vt
    return G

'''
def main():

    # Adjust to index conv layers in VGGs
    idxs = list(range(8))

    fname = 'csv_logs/test.csv'
    outf = open(fname, 'w')

    net = models.vgg11(weights="DEFAULT")
    #init_net is used as a reference untrained network.
    init_net = models.vgg11(weights=None)

    # Modules is unused.
    modules= list(net.children())[:-1]
    modules += [nn.Flatten(), list(net.children())[-1]]

    # Set path to imagenet data
    path = None

    trainloader, _ = dataset.get_imagenet(batch_size=2, path=path)

    for idx in idxs:
        i_val, r_val = verify_NFA(net, init_net, trainloader, layer_idx=idx)
        print("Layer " + str(idx+1) + ',' + str(i_val) + ',' + str(r_val), file=outf, flush=True)
'''
#TODO: ADD a visualizer for the image

#if __name__ == "__main__":
    #main()

Setting up a new session...
Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


'\ndef main():\n\n    # Adjust to index conv layers in VGGs\n    idxs = list(range(8))\n\n    fname = \'csv_logs/test.csv\'\n    outf = open(fname, \'w\')\n\n    net = models.vgg11(weights="DEFAULT")\n    #init_net is used as a reference untrained network.\n    init_net = models.vgg11(weights=None)\n\n    # Modules is unused.\n    modules= list(net.children())[:-1]\n    modules += [nn.Flatten(), list(net.children())[-1]]\n\n    # Set path to imagenet data\n    path = None\n\n    trainloader, _ = dataset.get_imagenet(batch_size=2, path=path)\n\n    for idx in idxs:\n        i_val, r_val = verify_NFA(net, init_net, trainloader, layer_idx=idx)\n        print("Layer " + str(idx+1) + \',\' + str(i_val) + \',\' + str(r_val), file=outf, flush=True)\n'

In [67]:
class PatchConvLayer(nn.Module):
    def __init__(self, conv_layer):
        super().__init__()
        self.layer = conv_layer #(k,c,q,s)

    def forward(self, patches):
        out = torch.einsum('nwhcqr, kcqr -> nwhk', patches, self.layer.weight)
        n, w, h, k = out.shape
        out = out.transpose(1, 3).transpose(2, 3) #(n,k,w_out,h_out)
        return out


layer_idx=0
count=0
# Get the layer_idx+1 th conv layer
for idx, m in enumerate(net.children()):
        if isinstance(m, P4ConvZ2):
            print("hi")
            count += 1
        if count-1 == layer_idx:
            l_idx = idx
            break
layers= list(net.children())

layer = layers[l_idx]
print(l_idx)
print(layer.weight)
print(layer.kernel_size)
print(layer.padding)
print(layer.stride)
# Extract W matrix
M = layer.weight
#M0= layer_init.weight
patchnet = deepcopy(net)
layer = PatchConvLayer(layers[l_idx])
patchnet= layers[l_idx:]
patchnet[0] = layer
print(patchnet)

#_, ki, q, s = M.shape
#print(M.shape)
#_,ki, q, s = M.shape

k, ki, ip_stab, q,s= M.shape
            
# Build W which is a (k, c*q*s) matrix. What to do with ip_stab
M = M.reshape(-1, ki*ip_stab*q*s)
            
# Compute WtW which is (c*q*s,c*q*s) matrix
M = torch.einsum('nd, nD -> dD', M, M)

M0=M
# Build W0 from the untrained net which is a (k, c*q*s) matrix         
M0 = M0.reshape(-1, ki*ip_stab*q*s)
    
# Compute W0tW0 which is (c*q*s,c*q*s) matrix
M0 = torch.einsum('nd, nD -> dD', M0, M0)

print(M)




hi
0
Parameter containing:
tensor([[[[[ 0.0292,  0.2141,  0.0165],
           [ 0.4765,  0.1134, -0.4281],
           [ 0.2660, -0.1225,  0.1569]]]],



        [[[[-0.3971,  0.3040, -0.0782],
           [-0.2174,  0.3545, -0.1848],
           [ 0.1032,  0.2699,  0.3735]]]],



        [[[[ 0.1157,  0.3254, -0.1751],
           [-0.1815, -0.2930, -0.2022],
           [ 0.1058, -0.1116, -0.3284]]]],



        [[[[ 0.2150, -0.0393, -0.3234],
           [ 0.4192, -0.2133, -0.1919],
           [ 0.4808,  0.4288, -0.2992]]]],



        [[[[ 0.0141,  0.2815, -0.0548],
           [ 0.2867, -0.1359, -0.3368],
           [ 0.0087, -0.2200,  0.1808]]]],



        [[[[ 0.4491, -0.2679, -0.2949],
           [ 0.0531,  0.1287, -0.3561],
           [ 0.2307, -0.2356, -0.3471]]]],



        [[[[-0.0144,  0.2857,  0.1793],
           [ 0.2572,  0.3503, -0.1913],
           [ 0.1426, -0.2798, -0.1957]]]],



        [[[[ 0.3386,  0.1300, -0.1025],
           [ 0.1340,  0.2924, -0.4710],
           

In [ ]:
net.eval()
net.cuda()
patchnet.eval()
patchnet.cuda()
M = 0
q, s = (3,3)
pad1, pad2 = (0,0)
s1, s2 = (1,1)
MAX_NUM_IMGS = 10
for idx, batch in enumerate(trainloader):
        print("Computing GOP for sample " + str(idx) + \
              " out of " + str(MAX_NUM_IMGS))
        imgs, _ = batch
        with torch.no_grad():
            imgs = imgs.cuda()        
            # Run the first half of the network wrt to the current layer 
            imgs = net.children()[:layer_idx](imgs).cpu() #(n,c,h,w)
        patches = patchify(imgs, (q, s), (s1,s2), padding=(pad1,pad2))#(n,w_out,h_out,c,q,s)
        patches = patches.cuda()
        #print(patches.shape)
        M += egop(patchnet, patches).cpu()
        del imgs, patches
        torch.cuda.empty_cache()
        if idx >= MAX_NUM_IMGS:
            break

In [86]:
import torch

def rotate_4_times(input_tensor):
  """
  Generates a tensor with four clockwise rotations of the last two dimensions.

  Args:
    input_tensor: A tensor of shape (k, c, q, r).

  Returns:
    A tensor of shape (k, c, q, 4, r) where the last two dimensions
    are rotated clockwise 0, 90, 180, and 270 degrees respectively.
  """
  k, c, q, r = input_tensor.shape
  rotated_tensor = torch.zeros(k, c, q, 4, r, dtype=input_tensor.dtype, device=input_tensor.device)

  for i in range(k):
    for j in range(c):
      slice_qr = input_tensor[i, j, :, :]
      rotated_tensor[i, j, :, 0, :] = slice_qr  # 0-degree rotation (original)

      rotated_90 = torch.rot90(slice_qr, k=-1, dims=(0, 1))
      if rotated_90.shape == (r, q):
          rotated_tensor[i, j, :q, 1, :r] = rotated_90.transpose(0, 1) # Transpose to (q, r)
      else:
          raise RuntimeError(f"Unexpected shape after 90-degree rotation: {rotated_90.shape}")

      rotated_180 = torch.rot90(slice_qr, k=-2, dims=(0, 1))
      rotated_tensor[i, j, :, 2, :] = rotated_180 # Shape will be (q, r)

      rotated_270 = torch.rot90(slice_qr, k=-3, dims=(0, 1))
      if rotated_270.shape == (r, q):
          rotated_tensor[i, j, :q, 3, :r] = rotated_270.transpose(0, 1) # Transpose to (q, r)
      else:
          raise RuntimeError(f"Unexpected shape after 270-degree rotation: {rotated_270.shape}")

  return rotated_tensor

# Example usage:
k, c, q, r = 2, 3, 5, 5
original_tensor = torch.randn(k, c, q, r)
rotated_4_tensor = rotate_4_times(original_tensor)

print("Original Tensor Shape:", original_tensor.shape)
print("Rotated 4 Times Tensor Shape:", rotated_4_tensor.shape)

print("\nOriginal slice (k=0, c=0):\n", original_tensor[0, 0])
print("\n0-degree rotated slice (k=0, c=0, rotation=0):\n", rotated_4_tensor[0, 0, :, 0, :])
print("\n90-degree rotated slice (k=0, c=0, rotation=1):\n", rotated_4_tensor[0, 0, :, 1, :])
print("\n180-degree rotated slice (k=0, c=0, rotation=2):\n", rotated_4_tensor[0, 0, :, 2, :])
print("\n270-degree rotated slice (k=0, c=0, rotation=3):\n", rotated_4_tensor[0, 0, :, 3, :])

Original Tensor Shape: torch.Size([2, 3, 5, 5])
Rotated 4 Times Tensor Shape: torch.Size([2, 3, 5, 4, 5])

Original slice (k=0, c=0):
 tensor([[ 1.9671,  1.9197,  0.4562,  2.3616, -1.6425],
        [ 0.4313, -1.1668, -2.9183, -1.4771, -0.3668],
        [ 1.2567,  0.7854, -1.8855,  0.5730, -1.4748],
        [-0.5583,  1.5608, -0.2936,  0.5151,  0.3046],
        [ 1.1858, -0.7286,  0.4793,  0.1773, -0.5086]])

0-degree rotated slice (k=0, c=0, rotation=0):
 tensor([[ 1.9671,  1.9197,  0.4562,  2.3616, -1.6425],
        [ 0.4313, -1.1668, -2.9183, -1.4771, -0.3668],
        [ 1.2567,  0.7854, -1.8855,  0.5730, -1.4748],
        [-0.5583,  1.5608, -0.2936,  0.5151,  0.3046],
        [ 1.1858, -0.7286,  0.4793,  0.1773, -0.5086]])

90-degree rotated slice (k=0, c=0, rotation=1):
 tensor([[ 1.1858, -0.7286,  0.4793,  0.1773, -0.5086],
        [-0.5583,  1.5608, -0.2936,  0.5151,  0.3046],
        [ 1.2567,  0.7854, -1.8855,  0.5730, -1.4748],
        [ 0.4313, -1.1668, -2.9183, -1.4771, -0.3